In [2]:
import pandas as pd
import warnings
from apyori import apriori
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('../Dataset/Dataset_EditActions.csv')
df = df.drop(['Architecture', 'Reproducible?', 'Code Snippet Present?', 'System Configuration Present?', 'Data Description Present?', 'Framework'], axis = 1)

In [4]:
# Replace Training Bug with value T, Model bug with value M, API bug with value A, Tensor & Input Bug with value I, and GPU Bug with value G
df.loc[df['Type of Bug'] == 'Training Bug', 'Type of Bug'] = 'T'
df.loc[df['Type of Bug'] == 'Model Bug', 'Type of Bug'] = 'M'
df.loc[df['Type of Bug'] == 'API Bug', 'Type of Bug'] = 'A'
df.loc[df['Type of Bug'] == 'Tensor and Input Bug', 'Type of Bug'] = 'I'
df.loc[df['Type of Bug'] == 'GPU Bug', 'Type of Bug'] = 'G'
df.loc[df['Type of Bug'] == 'Mixed Bug', 'Type of Bug'] = 'X'

In [5]:
# Map values from new columns as the following:
# Input Data Generation: D 
# Neural Network Definition: N
# Obsolete Parameter Removal: O
# Framework Migration: F
# Dataset Procurement: P
# Models and Tokenizers: M
# Logging: L
# Import Addition and Dependency Resolution: R
# Compiler Error Resolution: C
# Hyperparameter Initialization: H

# Replace all 1's with the corresponding letter
df['Input Data Generation'] = df['Input Data Generation'].replace(1, 'D')
df['Neural Network Definition'] = df['Neural Network Definition'].replace(1, 'N')
df['Obsolete Parameter Removal'] = df['Obsolete Parameter Removal'].replace(1, 'O')
df['Framework Migration'] = df['Framework Migration'].replace(1, 'F')
df['Dataset Procurement'] = df['Dataset Procurement'].replace(1, 'P')
df['Downloading Models and Tokenizers'] = df['Downloading Models and Tokenizers'].replace(1, 'M')
df['Logging'] = df['Logging'].replace(1, 'L')
df['Import Addition and Dependency Resolution'] = df['Import Addition and Dependency Resolution'].replace(1, 'R')
df['Compiler Error Resolution'] = df['Compiler Error Resolution'].replace(1, 'C')
df['Hyperparameter Initialization'] = df['Hyperparameter Initialization'].replace(1, 'H')


df['Input Data Generation'] = df['Input Data Generation'].replace(0, '')
df['Neural Network Definition'] = df['Neural Network Definition'].replace(0, '')
df['Obsolete Parameter Removal'] = df['Obsolete Parameter Removal'].replace(0, '')
df['Framework Migration'] = df['Framework Migration'].replace(0, '')
df['Dataset Procurement'] = df['Dataset Procurement'].replace(0, '')
df['Downloading Models and Tokenizers'] = df['Downloading Models and Tokenizers'].replace(0, '')
df['Logging'] = df['Logging'].replace(0, '')
df['Import Addition and Dependency Resolution'] = df['Import Addition and Dependency Resolution'].replace(0, '')
df['Compiler Error Resolution'] = df['Compiler Error Resolution'].replace(0, '')
df['Hyperparameter Initialization'] = df['Hyperparameter Initialization'].replace(0, '')

In [6]:
training_bugs = df[df['Type of Bug'] == 'T']
gpu_bugs = df[df['Type of Bug'] == 'G']
api_bugs = df[df['Type of Bug'] == 'A']
model_bugs = df[df['Type of Bug'] == 'M']
tensor_bugs = df[df['Type of Bug'] == 'I']
mixed_bugs = df[df['Type of Bug'] == 'X']

training_transactions = []
for i in range(0, len(training_bugs)):
    training_transactions.append([str(training_bugs.values[i,j]) for j in range(0, len(training_bugs.columns)) if str(training_bugs.values[i,j]) != ''])

gpu_transactions = []
for i in range(0, len(gpu_bugs)):
    gpu_transactions.append([str(gpu_bugs.values[i,j]) for j in range(0, len(gpu_bugs.columns)) if str(gpu_bugs.values[i,j]) != ''])

api_transactions = []
for i in range(0, len(api_bugs)):
    api_transactions.append([str(api_bugs.values[i,j]) for j in range(0, len(api_bugs.columns)) if str(api_bugs.values[i,j]) != ''])

model_transactions = []
for i in range(0, len(model_bugs)):
    model_transactions.append([str(model_bugs.values[i,j]) for j in range(0, len(model_bugs.columns)) if str(model_bugs.values[i,j]) != ''])

tensor_transactions = []
for i in range(0, len(tensor_bugs)):
    tensor_transactions.append([str(tensor_bugs.values[i,j]) for j in range(0, len(tensor_bugs.columns)) if str(tensor_bugs.values[i,j]) != ''])

mixed_transactions = []
for i in range(0, len(mixed_bugs)):
    mixed_transactions.append([str(mixed_bugs.values[i,j]) for j in range(0, len(mixed_bugs.columns)) if str(mixed_bugs.values[i,j]) != ''])

transactions = []
for i in range(0, len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0, len(df.columns)) if str(df.values[i,j]) != ''])

In [14]:
def get_apriori_results(rules, character):
    for result in list(rules):
     if character in result.items:
        items = ", ".join(result.items)
        support = result.support
        print(f"Items: {{{items}}}")
        print(f"Support: {support:.4f}")
        if result.ordered_statistics:
            print("Association Rules:")
            for rule in result.ordered_statistics:
                if character in rule.items_base:
                    antecedent = ", ".join(rule.items_base)
                    consequent = ", ".join(rule.items_add)
                    confidence = rule.confidence
                    lift = rule.lift
                    print(f"  {{{antecedent}}} => {{{consequent}}}")
                    print(f"Confidence: {confidence:.4f}")

In [15]:
get_apriori_results(apriori(training_transactions, max_length = 2), 'T')

57


In [16]:
get_apriori_results(apriori(model_transactions, max_length = 2), 'M')

185


In [17]:
get_apriori_results(apriori(gpu_transactions, max_length = 2), 'G')

1


In [18]:
get_apriori_results(apriori(tensor_transactions, max_length = 2), 'I')

63


In [19]:
get_apriori_results(apriori(mixed_transactions, max_length = 2), 'X')

91


In [20]:
get_apriori_results(apriori(api_transactions, max_length = 2), 'A')

27


In [21]:
from pyECLAT import ECLAT

def get_eclat_support(transactions):
    eclat_instance = ECLAT(data=pd.DataFrame(transactions), verbose=True)
    get_ECLAT_indexes, get_ECLAT_supports = eclat_instance.fit(separator='', verbose=True)
    print (len(get_ECLAT_supports))

In [22]:
get_eclat_support(training_transactions)

100%|██████████| 11/11 [00:00<00:00, 2749.05it/s]


Combination 1 by 1


10it [00:00, 213.51it/s]


Combination 2 by 2


45it [00:00, 238.75it/s]


Combination 3 by 3


120it [00:00, 449.11it/s]

55


In [23]:
get_eclat_support(gpu_transactions)

100%|██████████| 1/1 [00:00<00:00, 1005.59it/s]


Combination 1 by 1


1it [00:00, 322.84it/s]


Combination 2 by 2


0it [00:00, ?it/s]


Combination 3 by 3


0it [00:00, ?it/s]

1


In [24]:
get_eclat_support(model_transactions)

100%|██████████| 11/11 [00:00<00:00, 1833.39it/s]


Combination 1 by 1


10it [00:00, 172.89it/s]


Combination 2 by 2


45it [00:00, 226.50it/s]


Combination 3 by 3


120it [00:00, 215.36it/s]

112


In [25]:
get_eclat_support(api_transactions)

100%|██████████| 11/11 [00:00<00:00, 2156.45it/s]


Combination 1 by 1


0it [00:00, ?it/s]

10it [00:00, 247.91it/s]


Combination 2 by 2


45it [00:00, 233.23it/s]


Combination 3 by 3


120it [00:00, 315.21it/s]

59


In [26]:
get_eclat_support(tensor_transactions)

100%|██████████| 11/11 [00:00<00:00, 2197.85it/s]


Combination 1 by 1


9it [00:00, 194.85it/s]


Combination 2 by 2


36it [00:00, 216.27it/s]


Combination 3 by 3


84it [00:00, 225.18it/s]

73


In [27]:
get_eclat_support(mixed_transactions)

100%|██████████| 9/9 [00:00<00:00, 1500.23it/s]


Combination 1 by 1


8it [00:00, 145.90it/s]


Combination 2 by 2


28it [00:00, 153.59it/s]


Combination 3 by 3


56it [00:00, 149.01it/s]

62


In [28]:
get_eclat_support(transactions)

100%|██████████| 16/16 [00:00<00:00, 2282.54it/s]


Combination 1 by 1


13it [00:00, 204.30it/s]


Combination 2 by 2


78it [00:00, 304.89it/s]


Combination 3 by 3


286it [00:00, 582.40it/s]

63
